**Übung Mustererkennung** *WS 2022/23* -- *K. Brandenbusch,  Gernot A. Fink* -- *Technische Universität Dortmund, Lehrstuhl XII, Mustererkennung in eingebetteten Systemen*
___
# Aufgabe 3: Vektorquantisierung zur Initialisierung eines Verteilungsmodells

Statt einer Normalverteilung pro Klasse bietet es sich häufig an, mehrere Verteilungen vorzusehen. Dafür ist es notwendig, geeignete Verteilungen im Datenraum zu ermitteln.
Für eine Initialisierung dieser Verteilungen bietet es sich an, die Daten zu Clustern. Anschließend kann für jedes Cluster eine Verteilung geschätzt werden.

In dieser Aufgabe sollen Sie den Algorithmus von Lloyd zur Vektorquantieisrung implementieren. Optional können Sie zusätzlich auf den $k$-means-Algorithmus nach McQueen implementieren.
***

Zuerst müssen die benötigten Module importiert werden.

In [0]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

import sys
from matplotlib import cm
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import cdist

# Uebergeordneten Ordner zum Pfad hinzufuegen, damit das common Package importiert werden kann
if '..' not in sys.path:
    sys.path.append('..')

from common import visualization
from common.data_provider import DataProvider

---
## Visualisierung der Trainingsdaten

Laden Sie die Trainingsdaten (ohne Label) und visualisieren Sie die Trainingsdaten in einem Scatter-Plot. Plotten Sie zunächst alle Punkte farblos (`c='w', edgecolor='k'`).

---
## Initialisierung des Kodebuchs

Wählen Sie eine geeignete Kodebuchgröße und initialisieren Sie entsprechend viele zufällige Kodewörter.<br>
Hierzu können Sie die Methode [np.random.permutation](http://docs.scipy.org/doc/numpy/reference/generated/numpy.random.permutation.html) verwenden.

Visualisieren Sie anschließend die Kodewörter im zuvor erstellen Plot, indem Sie auf derselben axis erneut `scatter` aufrufen (der vorherige Plot wird dadurch aktualisiert).<br>
Geben Sie jedem Kodewort eine andere Farbe. Hierzu bietet sich die Verwendung einer [Colormap](https://matplotlib.org/stable/tutorials/colors/colormaps.html) an (z.B. `nipy_spectral` oder `gist_ncar`).

Warum ist es sinnvoll als Initialisierung zufällige Beispiele aus dem Datensatz auszuwählen?

---
## Zuordnung der Beispiele zu Kodewörtern

Ordnen Sie alle Beispiele einem Kodewort nach der **Nächster-Nachbar-Regel** zu und visualisieren Sie farblich die Zuordnung.

Nützliche Funktionen: 
- [scipy.distance.cdist](http://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html)
- [np.argmin](https://numpy.org/doc/stable/reference/generated/numpy.argmin.html)

---
## Berechnung des Quantisierungsfehlers

Berechnen Sie den Fehler der Zuordnung:

$\large \varepsilon = \sum^{L}_{\rho=1}\int_{V_\rho}|\underline{f}-\underline{b}_\rho|^2 p(\underline{f})d\underline{f} $

$\large L $ ist hier die Größe des Kodebuchs und $\large \underline{b}_\rho $ die Kodewörter.<br>
Überlegen Sie, wie Sie den Fehler basierend auf der Stichprobe berechnen können:
- Was sind die Vektoren $\large \underline{f} $ der Volumina $\large {V_\rho} $?
- Wie kann $\large p(\underline{f}) $ basierend auf der Stichprobe geschätzt werden?

Nützliche Funktionen:
- [np.min](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.min.html)
- [np.mean](https://numpy.org/doc/stable/reference/generated/numpy.mean.html)

---
## Aktualisierung der Kodewörter

Berechnen Sie das optimale Kodebuch für die aktuelle Zuordnung der Punkte.

Nützliche Funktion:
- [np.mean](https://numpy.org/doc/stable/reference/generated/numpy.mean.html)

---
## Iterative Optimierung

Legen Sie einen Threshold für die relative Änderung des Fehlers fest. Wiederholen Sie die bisherigen Schritte, um so nach dem Algorithmus von Lloyd den Quantisierungsfehler zu minimieren, bis der festgelegte Threshold unterschritten wird.<br>
Geben Sie nach jeder Iteration den Quantisierungsfehler aus.

Warum ist es sinnvoll die relative Änderung des Quantisierungsfehler als Abbruchkriterium zu verwenden?

Visualisieren Sie das berechnete Kodebuch und die Zuordnung der Daten in einem neuen Plot.

---
## Implementierung des vollständigen Algorithmus

Implementieren Sie nun die Methode `Lloyd.cluster()` in dem Modul [`common.vector_quantization`](../common/vector_quantization.py). Hierzu können Sie größtenteils Ihren bisherigen Code verwenden.<br>
Die Methode soll als Eingabe die Beispieldaten erhalten, welche Zeilenweise vorliegen. Zudem wird die Größe des Kodebuchs übergeben.<br>
Wenn das Flag  `verbose=True` gesetzt ist, soll jede Iteration der Quantisierungsfehler in der Konsole augegeben werden.

**Optional**: Die Methode `Lloyd.cluster()` akzeptiert zudem das Flag `prune_codebook`. Wenn dies gesetzt wurde, sollen alle Kodewörter verworfen werden, welchen nicht mehr Beispiele der Trainingsdaten zugeordnet werden, als die Daten Dimensionen besitzen.<br>
Implementieren Sie die Funktionalität dises Flags.


Warum kann diese Funktionalität für die später geplante Verwendung nützlich sein?

---

Importieren Sie die Klasse Lloyd und clustern Sie die Trainingsdaten. Visualisieren Sie anschliend das Ergebnis und vergleichen Sie es mit dem zuvor berechneten Ergebnis.

Warum können die Ergebnisse trotz korrekter Implementierung unterschiedlich sein?